# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# Name : Omer hazan

# Details Student 2:


## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_classif

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [ ]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
import wn
wordnet_he = wn.Wordnet('omw-he:1.4')

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
# YOUR CODE HERE
df_train.info() # no missing data


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   story   753 non-null    object
 1   gender  753 non-null    object
dtypes: object(2)
memory usage: 11.9+ KB


In [12]:
# Text Preprocessing:
def preprocess_story_text(text):
    old_text = text
    text = re.sub(r'\W+', ' ', text)     # Remove special characters
    text = re.sub(r'\d+', ' ', text)     # Remove numbers
    text = re.sub(r'\s\w\s', ' ', text)  # Cleanup single letters ex: "כ-3" will be "כ" after above
    text = re.sub(r'\s+', ' ', text)   # Remove extra spacing

    # if len(old_text) != len(text):
    #     print(f"{len(old_text) - len(text)}")
    #     print(f"{old_text}\n, {text}\n\n")

    return text

# df_train['story'] = df_train['story'].apply(preprocess_story_text)
# df_test['story'] = df_test['story'].apply(preprocess_story_text)

In [13]:
def expand_with_synonyms(text):
    expanded_text = []
    for word in text.split():
        synonyms = []
        for syn in wordnet_he.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma)
        if synonyms:
            expanded_text.extend(synonyms)
        else:
            expanded_text.append(word)  # Use the original word if no synonyms found
    return ' '.join(expanded_text)

# df_train['story'] = df_train['story'].apply(expand_with_synonyms)


In [14]:
# Text Preprocessing:
def preprocess_gender_text(text):
    text = re.sub(r'm', '1', text)     
    text = re.sub(r'f', '0', text)     

    # text = re.sub(r'm', '0', text)     
    # text = re.sub(r'f', '1', text)     
    return text

# df_train['gender'] = df_train['gender'].apply(preprocess_gender_text)

In [15]:
# TODO: create test preparation after tuning
def before_tuning_preparations_train(df_train:pd.DataFrame):
    # Split data
    X = df_train["story"]
    y = df_train["gender"]

    # Vectorize the data:
    vec = TfidfVectorizer(ngram_range = (1,1), min_df = 5, max_features = 2500)
    # vec = TfidfVectorizer(ngram_range = (1,2), min_df = 5, max_features=3000)
    # vec = CountVectorizer(max_features = 20)
    # vec = CountVectorizer(max_features = 50)
    # vec = CountVectorizer(max_features = 10)
    X_vec = vec.fit_transform(X)

    # Feature Selection:
    selector = SelectKBest(mutual_info_classif, k='all') 
    X_selected = selector.fit_transform(X_vec, np.zeros(X_vec.shape[0]))

    # Scaling:
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    X = scaler.fit_transform(X_selected.toarray())

    return X, y

# X, y = before_tuning_preparations_train(df_train)


In [16]:
def prepare_test_stories(df:pd.DataFrame):
    # Split data
    test_stories = df["story"]

    # Vectorize the data:
    vec = TfidfVectorizer(ngram_range = (1,1), min_df = 5, max_features = 2500)
    # vec = TfidfVectorizer(ngram_range = (1,2), min_df = 5, max_features=3000)
    test_stories_vec = vec.fit_transform(test_stories)

    # Feature Selection:
    selector = SelectKBest(mutual_info_classif, k='all') 
    test_stories_selected = selector.fit_transform(test_stories_vec, np.zeros(test_stories_vec.shape[0]))

    # Scaling:
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    test_stories = scaler.fit_transform(test_stories_selected.toarray())

    return test_stories

In [17]:
def find_best_decision_tree(X_train: pd.Series, y_train: pd.Series):
    model = DecisionTreeClassifier()
    # params =  {'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [None, 3, 5, 7, 9, 10]} got max_depth: 7
    params =  {'criterion': ['gini', 'entropy', 'log_loss'], 'max_depth': [None, 5, 6, 7, 8, 9, 10]} # max_dept: 8

    # cross validation
    grid_search = GridSearchCV(model, params, cv = 10, n_jobs = -1, refit=True, scoring="f1_macro")
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    return best_model, best_params

In [18]:
def find_best_logistic_regression(X_train: pd.Series, y_train: pd.Series):
    model = linear_model.LogisticRegression()
    # params = {'C': [0.1, 1, 5, 7, 10, 12, 15], 'penalty': ['l1', 'l2'], 'multi_class': ['ovr']} # got C=15
    # params = {'C': [15, 20 , 25], 'penalty': ['l1', 'l2'], 'multi_class': ['ovr']} got C=20
    # params = {'C': [19, 20 , 21], 'penalty': ['l1', 'l2'], 'multi_class': ['ovr']} got C=19
    # params = {'C': [17, 18, 19], 'penalty': ['l1', 'l2'], 'multi_class': ['ovr']}
    params = {'C': [0.1, 1, 5, 7, 10, 12, 15, 17, 20], 'penalty': ['l1', 'l2'], 'multi_class': ['ovr']} # got C=15

    # cross validation
    grid_search = GridSearchCV(model, params, cv = 10, n_jobs = -1, refit=True, scoring="f1_macro")
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    return best_model, best_params

In [19]:
# Didn't use 
def find_best_k_means(X_train: pd.Series, y_train: pd.Series):
    model = KMeans()
    params = {'n_clusters': [2], 'init': ['k-means++', 'random'], 'n_init': [15]}

    # cross validation
    grid_search = GridSearchCV(model, params, cv = 10, n_jobs = -1, refit=True, scoring="f1_macro")
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    return best_model, best_params

In [20]:
def find_best_naive_bayes(X_train: pd.Series, y_train: pd.Series):
   model = MultinomialNB()
   # params = {'alpha': [0.1, 1, 10], 'fit_prior': [True, False], 'class_prior': [None, [0.3, 0.7], [0.5, 0.5]]} got C = 0.1
   # params = {'alpha': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 1, 10], 'fit_prior': [True, False], 'class_prior': [None, [0.3, 0.7], [0.5, 0.5]]} got C=0.2
   # params = {'alpha': [0.17, 0.2, 0.225], 'fit_prior': [True, False], 'class_prior': [None, [0.3, 0.7], [0.5, 0.5]]} still C=0.2
   # params = {'alpha': [0.17, 0.2, 0.225], 'fit_prior': [True, False], 'class_prior': [None, [0.2, 0.8], [0.4, 0.6], [0.3, 0.7]]}
   # params = {'alpha': [0.17, 0.2, 0.225, 0], 'fit_prior': [True, False], 'class_prior': [None, [0.35, 0.65], [0.3, 0.7], [0.25, 0.75]]}
   # params = {'alpha': [0.17, 0.2, 0.225, 0.25, 0.3, 0.35, 0.4, 0.45], 'fit_prior': [True, False], 'class_prior': [None, [0.35, 0.65], [0.3, 0.7], [0.25, 0.75]]}
   # params = {'alpha': [0.3, 0.35, 0.4, 0.45], 'fit_prior': [True, False], 'class_prior': [None, [0.35, 0.65], [0.3, 0.7], [0.25, 0.75]]} # for ngram_range = (1,2)

   params = {'alpha': [0.1, 0.3, 0.5, 0.7, 0.9], 'fit_prior': [True, False], 'class_prior': [None, [0.3, 0.7], [0.5, 0.5]]}

   # cross validation
   grid_search = GridSearchCV(model, params, cv = 7, n_jobs = -1, refit=True, scoring="f1_macro")
   grid_search.fit(X_train, y_train)
   best_params = grid_search.best_params_
   best_model = grid_search.best_estimator_

   return best_model, best_params

In [21]:
def find_best_svm(X_train: pd.Series, y_train: pd.Series):
    model = LinearSVC()
    params = {'penalty': ['l1', 'l2'], 'loss': ['hinge', 'squared_hinge'], 'C': [0.1, 1, 10]} # got C=0.1
    # params = {'penalty': ['l1', 'l2'], 'loss': ['hinge', 'squared_hinge'], 'C': [0.05, 0.09, 0.1, 0.11, 0.15, 0.2]}
 
   # cross validation
    grid_search = GridSearchCV(model, params, cv = 10, n_jobs = -1, refit=True, scoring="f1_macro")
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    return best_model, best_params

In [22]:
def calc_and_display_scores(name, model, X: pd.Series, y: pd.Series, best_params):
    score = cross_val_score(model, X, y, scoring='f1_macro', cv=10, n_jobs=-1)
    print(f"***** {name} *****")
    print(best_params)
    print(score.mean()) 
    print("\n")


In [23]:
# Main Flow

# Text Cleanup
df_train['story'] = df_train['story'].apply(preprocess_story_text)

# Encode Gender
df_train['gender'] = df_train['gender'].apply(preprocess_gender_text)

# Adding synonyms words using `wordnet`
df_train['story'] = df_train['story'].apply(expand_with_synonyms)

# prepare data
X_train, y_train = before_tuning_preparations_train(df_train)

In [24]:

# Find best modal for each
best_decision_tree, dt_best_params = find_best_decision_tree(X_train, y_train)
# best_k_means = find_best_k_means(X_train, y_train)
best_logistic_regression, lr_best_params = find_best_logistic_regression(X_train, y_train)
best_naive_bayes, nb_best_params = find_best_naive_bayes(X_train, y_train)
best_svm, svm_best_params = find_best_svm(X_train, y_train)

# Evaluate model performance
calc_and_display_scores("Decision Tree", best_decision_tree, X_train, y_train, dt_best_params)
# calc_and_display_scores("K-Means", best_k_means, X_train, y_train)
calc_and_display_scores("Logistic Regression", best_logistic_regression, X_train, y_train, lr_best_params)
calc_and_display_scores("Naive Bayes", best_naive_bayes, X_train, y_train, nb_best_params)
calc_and_display_scores("SVM", best_svm, X_train, y_train, svm_best_params)

***** Decision Tree *****
{'criterion': 'entropy', 'max_depth': 9}
0.5980864082033912


***** Logistic Regression *****
{'C': 10, 'multi_class': 'ovr', 'penalty': 'l2'}
0.6465139439412381


***** Naive Bayes *****
{'alpha': 0.3, 'class_prior': None, 'fit_prior': False}
0.6897924212341574


***** SVM *****
{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}
0.6633420495234238




In [25]:
# Initial Scores before tuning:
# Dropping K-Means, can't fix bad results

# ***** Decision Tree *****
# {'criterion': 'gini', 'max_depth': 7}
# 0.5906157062274447

# ***** K-Means *****
# nan

# ***** Logistic Regression *****
# {'C': 10, 'multi_class': 'ovr', 'penalty': 'l2'}
# 0.5975029101388863

# ***** Naive Bayes *****
# {'alpha': 0.1, 'class_prior': None, 'fit_prior': False}
# 0.6010884801627505

# ***** SVM *****
# {'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}
# 0.6515963909551301


In [26]:
# tuning hyper parameters:

# ***** Decision Tree *****
# {'criterion': 'gini', 'max_depth': 8}
# 0.5937440736482061

# {'C': 17, 'multi_class': 'ovr', 'penalty': 'l2'}
# ***** Logistic Regression *****
# 0.6144839900244524

# {'alpha': 0.225, 'class_prior': [0.25, 0.75], 'fit_prior': True}
# ***** Naive Bayes *****
# 0.6389862182737506

# ***** SVM *****
# {'C': 0.1, 'loss': 'hinge', 'penalty': 'l2'}
# 0.6515963909551301

In [27]:
# Tried using StandardScaler:
# Not compatible with MultinomialNB and got worse results other than Decision Tree

# ***** Decision Tree *****
# {'criterion': 'gini', 'max_depth': 8}
# 0.6105151874513002

# ***** Logistic Regression *****
# {'C': 18, 'multi_class': 'ovr', 'penalty': 'l2'}
# 0.5320342069794771

# ***** SVM *****
# {'C': 0.05, 'loss': 'hinge', 'penalty': 'l2'}
# 0.23024308345336536

In [28]:
# Tuning ngram_range:
# vec = TfidfVectorizer(ngram_range = (1,2), min_df = 5)
# Naive Bayes improved a lot but the others decreased
# Other attempt to improve vectorization din't improve, didn't save result

# ***** Decision Tree *****
# {'criterion': 'log_loss', 'max_depth': 8}
# 0.5818965868866559

# ***** Logistic Regression *****
# {'C': 17, 'multi_class': 'ovr', 'penalty': 'l2'}
# 0.5888010588138272

# ***** Naive Bayes *****
# {'alpha': 0.3, 'class_prior': None, 'fit_prior': True}
# 0.6755624309104158

# ***** SVM *****
# {'C': 0.09, 'loss': 'hinge', 'penalty': 'l2'}
# 0.635969203632285

In [29]:
# After using wordnet
# ***** Decision Tree *****
# {'criterion': 'log_loss', 'max_depth': 10}
# 0.6122525866078513


# ***** Logistic Regression *****
# {'C': 10, 'multi_class': 'ovr', 'penalty': 'l2'}
# 0.6465139439412381


# ***** Naive Bayes *****
# {'alpha': 0.3, 'class_prior': None, 'fit_prior': False}
# 0.6897924212341574


# ***** SVM *****
# {'C': 1, 'loss': 'hinge', 'penalty': 'l2'}
# 0.6672685019921027

In [30]:
best_decision_tree, dt_best_params = find_best_decision_tree(X_train, y_train)
best_logistic_regression, lr_best_params = find_best_logistic_regression(X_train, y_train)
best_naive_bayes, nb_best_params = find_best_naive_bayes(X_train, y_train)
best_svm, svm_best_params = find_best_svm(X_train, y_train)

# evaluate model performance
calc_and_display_scores("Decision Tree", best_decision_tree, X_train, y_train, dt_best_params)
calc_and_display_scores("Logistic Regression", best_logistic_regression, X_train, y_train, lr_best_params)
calc_and_display_scores("Naive Bayes", best_naive_bayes, X_train, y_train, nb_best_params)
calc_and_display_scores("SVM", best_svm, X_train, y_train, svm_best_params)

***** Decision Tree *****
{'criterion': 'entropy', 'max_depth': 10}
0.6169629682571441


***** Logistic Regression *****
{'C': 10, 'multi_class': 'ovr', 'penalty': 'l2'}
0.6465139439412381


***** Naive Bayes *****
{'alpha': 0.3, 'class_prior': None, 'fit_prior': False}
0.6897924212341574


***** SVM *****
{'C': 10, 'loss': 'squared_hinge', 'penalty': 'l2'}
0.6657075647607925




In [31]:
# Main Flow - moving forward with Naive Bayes

# Prepare Test Data:

df_test['story'] = df_test['story'].apply(preprocess_story_text)
df_test['story'] = df_test['story'].apply(expand_with_synonyms)

stories = prepare_test_stories(df_test)


In [39]:
df_predicted = pd.DataFrame(columns = ['test_example_id','predicted_category'])
prediction = best_naive_bayes.predict(stories)
score = cross_val_score(best_naive_bayes, X_train, y_train, scoring='f1_macro', cv=10, n_jobs=-1).mean()
print(f"Macro f1 score: {score:.3f}\n") 
df_predicted['test_example_id'] = df_test['test_example_id']
df_predicted['predicted_category'] = np.where(prediction == '1', 'm', 'f')
print("First 5 predictions:")
df_predicted.head(5)
print("Last 5 predictions:")
df_predicted.tail(5)
print("Predicated ratio:")
df_predicted['predicted_category'].value_counts(normalize=True)
print("Train set ratio:")
df_train['gender'].value_counts(normalize=True)


Macro f1 score: 0.690

First 5 predictions:


,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f


Last 5 predictions:


,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


Predicated ratio:


predicted_category
m   0.932
f   0.068
Name: proportion, dtype: float64

Train set ratio:


gender
1   0.764
0   0.236
Name: proportion, dtype: float64

In [33]:
# Even though naive bayes achieved a better f1 score, the ratio of male/female is off (assuming roughly even split)
# the problem seems to be over-fitting, I tried solving this issue but it didn't work
# moving forward with SVM, second best model and getting closer ration to train set:

df_predicted = pd.DataFrame(columns = ['test_example_id','predicted_category'])
prediction = best_svm.predict(stories)
score = cross_val_score(best_svm, X_train, y_train, scoring='f1_macro', cv=10, n_jobs=-1).mean()

print(f"Macro f1 score: {score:.3f}\n") 
df_predicted['test_example_id'] = df_test['test_example_id']
df_predicted['predicted_category'] = np.where(prediction == '1', 'm', 'f')
print("First 5 predictions:")
df_predicted.head(5)
print("Last 5 predictions:")
df_predicted.tail(5)
print("Predicated ratio:")
df_predicted['predicted_category'].value_counts(normalize=True)
print("Train set ratio:")
df_train['gender'].value_counts(normalize=True)



0.668


,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,f
4,4,f


,test_example_id,predicted_category
318,318,f
319,319,m
320,320,m
321,321,m
322,322,m


Predicated ratio:


predicted_category
m   0.842
f   0.158
Name: proportion, dtype: float64

Train set ratio:


gender
1   0.764
0   0.236
Name: proportion, dtype: float64

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [34]:
df_predicted.to_csv('classification_results.csv',index=False)